## CHSH game

### Alice & Bob vs. Charlie
1. Charile sends to Alice $x\in\{0,1\}$, and also sends to Bob $y\in\{0,1 \}$.
2. After Alice has received $x$, she returns $a$.
3. After Bob has received $y$, he returns $b$.
4. If $x\land y = a\otimes b$ is satisfied,  Alice and Bob win, otherwise they loses

### Win rate $S$
- classical case : $S\le0.75$
- quantum case : $S\gt0.75$ is possible

In [1]:
import random
import qiskit
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit import Aer, quantum_info

In [2]:
def classical(trials):
    S = 0
    
    for _ in range(trials):
        
        # random bits by Charlie(x,y)
        x = random.randint(0,1)
        y = random.randint(0,1)
        
        # responce by Alice
        a = 0
        
        # responce by Bob
        b = 0
        
        # count up if Alice and Bob win
        if (x and y) == (a+b)%2:
            S = S + 1
        else:
            pass

    S /= trials
        
    print("S =",S)

In [3]:
trials = 1000
classical(trials)

S = 0.736


In [7]:
def quantum(trials):
    S = 0
    
    for _ in range(trials):
        
        # random bits by Charlie(x,y)
        x = random.randint(0,1)
        y = random.randint(0,1)
        
        # Bell state
        backend = Aer.get_backend('qasm_simulator')
        qc = QuantumCircuit(2)
        
        if x==0:    
            qc.h(0)
            qc.cx(0,1)
            qc.measure_all() # Z-basis measure
            job = backend.run(qc,shots=1)
            count = job.result().get_counts()
            if count.get('00')==1 or count.get('10')==1:
                a=0
            else:
                a=1
        elif x==1:
            qc.h(0)
            qc.cx(0,1)
            qc.h(0) # X-basis measure
            qc.measure_all()
            job = backend.run(qc,shots=1)
            count = job.result().get_counts()
            if count.get('00')==1 or count.get('10')==1:
                a=0
            else:
                a=1  
        else:
            pass
        
        if y==0:
            qc.h(0)
            qc.cx(0,1)
            qc.ry(0.25*np.pi, 1)
            qc.measure_all()
            job = backend.run(qc,shots=1)
            count = job.result().get_counts()
            if count.get('00')==1 or count.get('01')==1:
                b=0
            else:
                b=1
        elif y==1:
            qc.h(0)
            qc.cx(0,1)
            qc.ry(-0.25*np.pi, 1)
            qc.measure_all()
            job = backend.run(qc,shots=1)
            count = job.result().get_counts()
            if count.get('00')==1 or count.get('01')==1:
                b=0
            else:
                b=1
        else:
            pass
                
        # count up if Alice and Bob win
        if (x and y) == (a+b)%2:
            S = S + 1
        else:
            pass

    S /= trials
        
    print("S =",S)

In [8]:
trials = 1000
quantum(trials)

S = 0.505


In [6]:
import qiskit.tools.jupyter
%qiskit_version_table